# Framing Prediction Problem

In [204]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
pd.options.plotting.backend = 'plotly'
from itertools import chain

from utils.eda import *
from utils.dsc80_utils import *
from utils.graph import *

from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize

In [205]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
interactions = pd.read_csv('food_data/RAW_interactions.csv')
recipes = pd.read_csv('food_data/RAW_recipes.csv')
step0 = recipes.merge(interactions, how='left', left_on='id', right_on='recipe_id', indicator=True)
df_recipe = (step0
      .pipe(initial)
      .pipe(transform_df)
      #.pipe(outlier)
      .pipe(group_recipe)
      #.pipe(group_user)
)

# Problem Identification
**Analysis**:
Identify a prediction problem. Feel free to use one of the example prediction problems stated in the “Example Questions and Prediction Problems” section of your dataset’s description page or pose a hypothesis test of your own. The prediction problem you come up with doesn’t have to be related to the question you were answering in Steps 1-4, but ideally, your entire project has some sort of coherent theme.

**Report**:
Clearly state your prediction problem and type (classification or regression). If you are building a classifier, make sure to state whether you are performing binary classification or multiclass classification. Report the response variable (i.e. the variable you are predicting) and why you chose it, the metric you are using to evaluate your model and why you chose it over other suitable metrics (e.g. accuracy vs. F1-score).

Note: Make sure to justify what information you would know at the “time of prediction” and to only train your model using those features. For instance, if we wanted to predict your final exam grade, we couldn’t use your Project 4 grade, because Project 4 is only due after the final exam! Feel free to ask questions if you’re not sure.

### Some Potential Ideas:
1. Sentiment Analysis with `review` column
2. Using   `recipe` column and feature engineering (length of `recipe`, TF-IDF, ...) to predict `ratings`
3. Using text data as a input to predict the rating of the user and identify preference of users (pre-step to reconmender system)

In [221]:
df_recipe.columns

Index(['minutes', 'n_steps', 'n_ingredients', 'avg_rating', 'rating',
       'calories', 'total_fat', 'sugar', 'sodium', 'protein', 'sat_fat',
       'carbs', 'steps', 'name', 'description', 'ingredients', 'user_id',
       'contributor_id', 'review_date', 'review', 'recipe_date', 'tags'],
      dtype='object')

#  Framing Question
We know that Recipe's mean TFIDF distribution is different for higher rating recipe than lower rating recipe:
- We need `X` and a `y` -> find relationships! -> Supervised ML model
- We currently have the DataFrame grouped by recipe
- We want to predict `rating` as a classfication problem
    - `rating` in recipe df: a quality of recipe
    - `rating` in user_id df: user preference ✅
- Features for user_id df:
    - `TF-IDF mean/max/sum/partial_mean` of `description` for **recipe per user_id** (may have more than one recipe) that have **high ratings**
        - This evaluates whether a word shows more often in this **user's high rated recipe decription** compare to all **recipe decription**, thus, meaning that it is more important to this user.
    - `n_ingredients`
    - `n_steps`
    - `minutes`
    - `calories`
    - `sodium`
    - `previous_rating` (need to explore)
    - `word2vec` (need to explore, somr info [here](https://towardsdatascience.com/word2vec-explained-49c52b4ccb71)) 
        - Each `user_id` have a pool of words in a **vector space** (from description, can have more)
        - We want to see how similar (cosine distance) between recipe tags `word2vec` and the pool
        - [good theory background](https://medium.com/@zafaralibagh6/a-simple-word2vec-tutorial-61e64e38a6a1)

- consider using `tags`, `review`, `steps`?

- Further: using preference to recomand recipe!

- `Voting`?

- [Gaussian Bayesian Network](https://scikit-learn.org/stable/modules/naive_bayes.html)?

# Baseline Model
### Baseline Model
**Analysis**:
Train a “baseline model” for your prediction task that uses at least two features. (For this requirement, two features means selecting at least two columns from your original dataset that you should transform). You can leave numerical features as-is, but you’ll need to take care of categorical columns using an appropriate encoding. Implement all steps (feature transforms and model training) in a single sklearn Pipeline.

Note: Both now and in Step 7: Final Model, make sure to evaluate your model’s ability to generalize to unseen data!

There is no “required” performance metric that your baseline model needs to achieve.

**Report**:
Describe your model and state the features in your model, including how many are quantitative, ordinal, and nominal, and how you performed any necessary encodings. Report the performance of your model and whether or not you believe your current model is “good” and why.

Tip: Make sure to hit all of the points above: many projects in the past have lost points for not doing so.

## Pipeline
### Predictive Question
We want to predict `rating` as a classfication problem, prdicting `rating` (5 catagories) in the user_id DataFrame to demonstarte understanding of user preference.
- Using the original big DataFrame for predicting rating

### Feature Engineering
Remanber to take care of the missing data

**Feature Direct Usage**
- `n_ingredients`
- `n_steps`
- `minutes`
- `calories`
- `sodium`

**Feature Engineering**
- `tfidf_10_partial_mean` of `description` for **recipe per user_id** (may have more than one recipe) that have **high ratings**
    - This evaluates whether a word shows more often in this **user's high rated recipe decription** compare to all **recipe decription**, thus, meaning that it is more important to this user.
- `Word2Vec` Similarity
    - All good `recipe` (above 3 rating) can be a pool of words in a **vector space** (from description, can have more)
    - We want to see how similar (cosine distance) between each recipe `word2vec` description's vector to the good pool of vectors

### Ensemble Learning (Bagging, Stacking, Boosting)
Heterogenous Ensemble Voting:
1. Homogenous Ensemble `Rabndom Forest`
2. Model2...
3. Model3...

In [222]:
base_df = (step0.pipe(initial).pipe(transform_df))
base_df['description']

0         these are the most; chocolatey, moist, rich, d...
1         this is the recipe that we use at my school ca...
2         since there are already 411 recipes for brocco...
                                ...                        
234426    i've heard of the 'cookies by design' company,...
234427    i've heard of the 'cookies by design' company,...
234428    i've heard of the 'cookies by design' company,...
Name: description, Length: 234429, dtype: object

In [225]:
display_df(base_df)

,name,minutes,contributor_id,recipe_date,...,sodium,protein,sat_fat,carbs
0,1 brownies in the world best ever,40,985201,2008-10-27,...,3.0,3.0,19.0,6.0
1,1 in canada chocolate chip cookies,45,1848091,2011-04-11,...,22.0,13.0,51.0,26.0
2,412 broccoli casserole,40,50969,2008-05-30,...,32.0,22.0,36.0,3.0
...,...,...,...,...,...,...,...,...,...
234426,cookies by design sugar shortbread cookies,20,506822,2008-04-15,...,4.0,4.0,11.0,6.0
234427,cookies by design sugar shortbread cookies,20,506822,2008-04-15,...,4.0,4.0,11.0,6.0
234428,cookies by design sugar shortbread cookies,20,506822,2008-04-15,...,4.0,4.0,11.0,6.0


In [224]:
display_df(df_recipe)

,minutes,n_steps,n_ingredients,avg_rating,...,review_date,review,recipe_date,tags
recipe_id,,,,,,,,,
275022.0,50.0,11.0,7.0,3.0,...,"[2008-04-07 00:00:00, 2013-06-07 00:00:00, 201...",[Easy comfort food! I definitely thought it wa...,"[2008-01-01 00:00:00, 2008-01-01 00:00:00, 200...","[60-minutes-or-less, time-to-make, course, mai..."
275024.0,55.0,6.0,8.0,3.0,...,[2009-05-26 00:00:00],[When I found myself needing a dessert and hav...,[2008-01-01 00:00:00],"[60-minutes-or-less, time-to-make, course, pre..."
275026.0,45.0,7.0,9.0,3.0,...,"[2013-09-21 00:00:00, 2013-12-17 00:00:00]","[Sorry, this one didn&#039;t work out so well....","[2008-01-01 00:00:00, 2008-01-01 00:00:00]","[60-minutes-or-less, time-to-make, course, mai..."
...,...,...,...,...,...,...,...,...,...
537543.0,55.0,8.0,10.0,NaN,...,[2018-12-15 00:00:00],[I Didn't see pumpkin puree in ingredients jus...,[2018-11-16 00:00:00],"[60-minutes-or-less, time-to-make, course, cui..."
537671.0,135.0,54.0,12.0,NaN,...,[2018-12-15 00:00:00],"[These are an absolute work of art, but just t...",[2018-11-28 00:00:00],"[time-to-make, course, preparation, occasion, ..."
537716.0,40.0,12.0,13.0,5.0,...,[2018-12-06 00:00:00],[These were the best game day sandwiches. I co...,[2018-12-04 00:00:00],"[60-minutes-or-less, time-to-make, course, mai..."


In [210]:
recipe_high_score_corpus = df_recipe[df_recipe['rating']>=3]['description'].astype(str).str.split(' ').to_list()
full_training_corpus = word_tokenize(' '.join(df_recipe[df_recipe['rating']>=3]['description'].astype(str)))

In [211]:
# Train a feature model
word_vec = Word2Vec(recipe_high_score_corpus, window=7, sg=1, min_count=3)

In [212]:
# base_df['description'].apply(lambda x: [word_vec.wv[word] for word in x if word in full_training_corpus.split(' ')])

In [213]:
check_words = base_df['description'].astype(str).apply(word_tokenize)

In [214]:
check_words

0         [these, are, the, most, ;, chocolatey, ,, mois...
1         [this, is, the, recipe, that, we, use, at, my,...
2         [since, there, are, already, 411, recipes, for...
                                ...                        
234426    [i, 've, heard, of, the, 'cookies, by, design,...
234427    [i, 've, heard, of, the, 'cookies, by, design,...
234428    [i, 've, heard, of, the, 'cookies, by, design,...
Name: description, Length: 234429, dtype: object

In [215]:
word_vec_num = check_words.apply(lambda x: [word_vec.wv[word] for word in x if word in word_vec.wv.key_to_index.keys()])

In [216]:
word_vec_num

0         [[-0.22415169, 0.14288595, -0.115010485, -0.15...
1         [[-0.12147238, 0.22682993, 0.121205576, -0.404...
2         [[0.19034623, -0.16149454, 0.32498103, 0.01796...
                                ...                        
234426    [[-0.023993827, 0.05469277, 0.16401625, -0.150...
234427    [[-0.023993827, 0.05469277, 0.16401625, -0.150...
234428    [[-0.023993827, 0.05469277, 0.16401625, -0.150...
Name: description, Length: 234429, dtype: object

In [217]:
vocab_sample = np.random.choice(list(word_vec.wv.key_to_index.keys()), 1000, replace=False)

In [218]:
vocab_sample

array(['shortbread', 'honeycrisp', 'combinations,', ..., 'states.',
       'plum', 'mugs'], dtype='<U51')

In [219]:
def calculate_similar(list):
    avg_similarities = []
    for word in list:
        similarities = [word_vec.wv.similarity(word, vocab_word) for vocab_word in vocab_sample if word in word_vec.wv.key_to_index.keys()]
        avg_similarities.append(np.mean(similarities))
    
    return pd.Series(avg_similarities).fillna(0).mean()

In [220]:
# check_words.apply(lambda x: calculate_similar(x))